# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns
import matplotlib as plt
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
import re
import os


%matplotlib inline

In [21]:
sns.set_theme(
    style = 'whitegrid'
    )

In [6]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []
stars = []
country= []
date= []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
        
    for para in parsed_content.find_all("div", class_ = "rating-10"):
        stars.append(para.span.get_text())
        
    for para in parsed_content.find_all("h3"):
        country.append(para.span.next_sibling.get_text().strip("( )"))
        
        
    for para in parsed_content.find_all('time'):
        date.append(para.get_text())
                
  
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [22]:
print(len(reviews))
print(len(country))
print(len(stars))
len(date)


1000
1000
1010


1000

### Create DataFrame

In [23]:
df = pd.DataFrame({"reviews":reviews,"stars": stars[0:1000], "date":date, "country": country})

In [24]:
df

,reviews,stars,date,country
0,✅ Trip Verified | My family and I have flown ...,\n\t\t\t\t\t\t\t\t\t\t\t\t\t5,9th July 2023,United Kingdom
1,✅ Trip Verified | This has been by far the wo...,4,9th July 2023,United States
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,8th July 2023,United Kingdom
3,Not Verified | BA is not treating its premium ...,2,6th July 2023,United Kingdom
4,✅ Trip Verified | 24 hours before our departu...,4,5th July 2023,South Africa
...,...,...,...,...
995,Not Verified | Glasgow to Miami via London. G...,2,7th September 2018,United Kingdom
996,✅ Trip Verified | London Heathrow to Budapest...,10,6th September 2018,United Kingdom
997,✅ Trip Verified | Budapest to London Heathrow...,7,5th September 2018,United Kingdom
998,✅ Trip Verified | London to Toronto. Group of...,6,3rd September 2018,Canada


In [25]:
df['stars'] = df['stars'].str.strip('\n \t')
df

,reviews,stars,date,country
0,✅ Trip Verified | My family and I have flown ...,5,9th July 2023,United Kingdom
1,✅ Trip Verified | This has been by far the wo...,4,9th July 2023,United States
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,8th July 2023,United Kingdom
3,Not Verified | BA is not treating its premium ...,2,6th July 2023,United Kingdom
4,✅ Trip Verified | 24 hours before our departu...,4,5th July 2023,South Africa
...,...,...,...,...
995,Not Verified | Glasgow to Miami via London. G...,2,7th September 2018,United Kingdom
996,✅ Trip Verified | London Heathrow to Budapest...,10,6th September 2018,United Kingdom
997,✅ Trip Verified | Budapest to London Heathrow...,7,5th September 2018,United Kingdom
998,✅ Trip Verified | London to Toronto. Group of...,6,3rd September 2018,Canada


In [38]:
df['verified']  = df.reviews.str.contains('Trip Verified')

In [39]:
df

,reviews,stars,date,country,verified
0,✅ Trip Verified | My family and I have flown ...,5,9th July 2023,United Kingdom,True
1,✅ Trip Verified | This has been by far the wo...,4,9th July 2023,United States,True
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,8th July 2023,United Kingdom,True
3,Not Verified | BA is not treating its premium ...,2,6th July 2023,United Kingdom,False
4,✅ Trip Verified | 24 hours before our departu...,4,5th July 2023,South Africa,True
...,...,...,...,...,...
995,Not Verified | Glasgow to Miami via London. G...,2,7th September 2018,United Kingdom,False
996,✅ Trip Verified | London Heathrow to Budapest...,10,6th September 2018,United Kingdom,True
997,✅ Trip Verified | Budapest to London Heathrow...,7,5th September 2018,United Kingdom,True
998,✅ Trip Verified | London to Toronto. Group of...,6,3rd September 2018,Canada,True


In [40]:
df.dtypes

reviews     object
stars       object
date        object
country     object
verified      bool
dtype: object

In [43]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()

corpus =[]
reviews_data = df.reviews.str.strip("✅ Trip Verified |")

for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [46]:
df['corpus'] = corpus

In [51]:
df.head()

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | My family and I have flown ...,5,9th July 2023,United Kingdom,True,family flown mostly british airway last year p...
1,✅ Trip Verified | This has been by far the wo...,4,9th July 2023,United States,True,far worst service plane obvious flying economy...
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,8th July 2023,United Kingdom,True,nov booked paid return journey new zealand ret...
3,Not Verified | BA is not treating its premium ...,2,6th July 2023,United Kingdom,False,verified ba treating premium economy passenger...
4,✅ Trip Verified | 24 hours before our departu...,4,5th July 2023,South Africa,True,hour departure ba cape town heathrow thursday ...


In [53]:
df.date= pd.to_datetime(df.date)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [54]:
df.dtypes

reviews             object
stars               object
date        datetime64[ns]
country             object
verified              bool
corpus              object
dtype: object

In [56]:
df.head()

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | My family and I have flown ...,5,2023-07-09,United Kingdom,True,family flown mostly british airway last year p...
1,✅ Trip Verified | This has been by far the wo...,4,2023-07-09,United States,True,far worst service plane obvious flying economy...
2,✅ Trip Verified | In Nov 2022 I booked and pa...,2,2023-07-08,United Kingdom,True,nov booked paid return journey new zealand ret...
3,Not Verified | BA is not treating its premium ...,2,2023-07-06,United Kingdom,False,verified ba treating premium economy passenger...
4,✅ Trip Verified | 24 hours before our departu...,4,2023-07-05,South Africa,True,hour departure ba cape town heathrow thursday ...


In [57]:
df.isnull().sum()

reviews     0
stars       0
date        0
country     0
verified    0
corpus      0
dtype: int64